In [2]:
print(ord("A") % 5 + 1)

1


In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import colors

def load_data(file_path, variant_number):
    df = pd.read_excel(file_path)
    data = df.iloc[variant_number - 1] 
    return data

def crop_image_around_face(image, face, target_size):
    x, y, w, h = face
    face_midpoint = (x + w // 2, y + h // 2)
    target_width, target_height = target_size

    left = max(0, face_midpoint[0] - target_width // 2)
    right = min(image.shape[1], face_midpoint[0] + target_width // 2)

    top = max(0, face_midpoint[1] - target_height // 2)
    bottom = min(image.shape[0], face_midpoint[1] + target_height // 2)

    crop_width = right - left
    if crop_width < target_width:
        if left == 0:
            right = min(right + (target_width - crop_width), image.shape[1])
        elif right == image.shape[1]:
            left = max(left - (target_width - crop_width), 0)

    crop_height = bottom - top
    if crop_height < target_height:
        if top == 0:
            bottom = min(bottom + (target_height - crop_height), image.shape[0])
        elif bottom == image.shape[0]:
            top = max(top - (target_height - crop_height), 0)

    return image[top:bottom, left:right]

def add_glasses(image_path, output_path, glasses_color, line_thickness, size):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye_tree_eyeglasses.xml')
        
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = img[y:y+h, x:x+w]
        img = crop_image_around_face(img, (x, y, w, h), target_size=size)
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.2, 4)
        if len(eyes) >= 2:
            eyes = sorted(eyes, key=lambda ex: ex[0])
            eye_1, eye_2 = eyes[:2]

            ex1, ey1, ew1, eh1 = eye_1
            ex2, ey2, ew2, eh2 = eye_2
            eye_center_1 = (ex1 + ew1 // 2, ey1 + eh1 // 2)
            eye_center_2 = (ex2 + ew2 // 2, ey2 + eh2 // 2)

            radius = max(ew1,ew2) // 2

            roi_color = cv2.circle(roi_color, eye_center_1, radius, glasses_color, line_thickness)
            roi_color = cv2.circle(roi_color, eye_center_2, radius, glasses_color, line_thickness)
            roi_color = cv2.line(roi_color,
                     (eye_center_1[0] + radius, eye_center_1[1]),
                     (eye_center_2[0] - radius, eye_center_2[1]),
                     glasses_color, line_thickness)

            roi_color = cv2.line(roi_color, 
                     (eye_center_1[0] - radius, eye_center_1[1]), 
                     (eye_center_1[0] - radius - (w // 4), eye_center_1[1]), 
                     glasses_color, line_thickness)

            roi_color = cv2.line(roi_color,(eye_center_2[0] + radius, eye_center_2[1]),
                     (x + w, eye_center_2[1]), glasses_color, line_thickness)

    cv2.imwrite(output_path, img)


variant_number = 5
data = load_data('./Lab5_ua/lab5.xlsx', variant_number)

filename = data['file name']
glasses_color = data['glasses color']
line_width = int(data['line width'])
size = map(int, data['image size'].split('x'))

color = colors.to_rgb(glasses_color)[::-1]
color = tuple(int(c * 255) for c in color)
add_glasses(f'./Lab5_ua/Images/{filename}', 'output_image5.jpg', glasses_color=color, line_thickness=line_width, size=size)